In [8]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [91]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
# options.add_argument("--disable-extensions")
# options.add_argument("disable-gpu")



In [113]:
camping = pd.read_csv("../file/camping.csv")

In [186]:
df1 = camping[['name','admin_code','collect_date','num_site']].groupby(['name','admin_code']).agg(['count','max']).reset_index()
df1.columns = ['_'.join(col).strip() for col in df1.columns.values]

print(len(df1))

print(df1.columns)
df1.head()
# df.groupby('Data').agg(['count','max','min','sum','mean'])
df1.columns = ['name','admin_code','cnt','max_date','ddd','max_site']
df1 = df1.loc[:, df1.columns != 'ddd']

df1[df1['cnt']==2].head()

2131
Index(['name_', 'admin_code_', 'collect_date_count', 'collect_date_max',
       'num_site_count', 'num_site_max'],
      dtype='object')


,name,admin_code,cnt,max_date,max_site
28,HJ글램핑,5600000,2,2019-09-09,46
200,광주시민의숲 야영장,3620000,2,2019-09-25,38
241,국립유명산자연휴양림,4160000,2,2019-09-08,92
400,다솔관광농원,4140000,2,2019-09-24,50
410,단호샌드파크 캠핑장,5070000,2,2019-10-15,23


In [187]:
camping[camping['name']=='HJ글램핑']

,name,name_modify,category,y_coord,x_coord,address_str,address_jibun,tel,num_site,area,...,conv,safety,etc_bldg,time,price,tel_admin,admin,collect_date,admin_code,admin_name
1199,HJ글램핑,HJ글램핑,야영장,38.059769,127.387582,경기도 포천시 이동면 화동로 2440-34,NaN,031-000-0000,46,NaN,...,"화장실, 샤워실, 매점",소화기,NaN,NaN,NaN,031-538-3364,경기도 포천시청(관광산업과),2019-09-09,5600000,경기도 포천시
1258,HJ글램핑,HJ글램핑,야영장,38.078596,127.389548,경기도 포천시 이동면 금강로 6169,경기도 포천시 이동면 도평리 512-4,031-000-0000,12,NaN,...,"화장실, 샤워실, 매점",소화기,NaN,NaN,NaN,031-538-3364,경기도 포천시청(관광산업과),2019-09-09,5600000,경기도 포천시


In [188]:
camping_f = pd.merge(df1, camping, how='inner', 
                     left_on=['name','admin_code','max_date', 'max_site'], 
                     right_on = ['name','admin_code','collect_date','num_site'],
                          sort=False)
print(len(camping_f))
camping_f[camping_f['name']=='HJ글램핑']

2154


,name,admin_code,cnt,max_date,max_site,name_modify,category,y_coord,x_coord,address_str,...,parking,conv,safety,etc_bldg,time,price,tel_admin,admin,collect_date,admin_name
28,HJ글램핑,5600000,2,2019-09-09,46,HJ글램핑,야영장,38.059769,127.387582,경기도 포천시 이동면 화동로 2440-34,...,NaN,"화장실, 샤워실, 매점",소화기,NaN,NaN,NaN,031-538-3364,경기도 포천시청(관광산업과),2019-09-09,경기도 포천시


In [104]:
camping_filter = pd.merge(camping, df1, on=['name','admin_code','collect_date'], how='inner', sort=False, suffixes='ab')
# camping_filter = camping.loc[camping['name'].isin(df1['name']) &
#             camping['admin_code'].isin(df1['admin_code']) &
#            camping['collect_date'].isin(df1['collect_date'])]
camping_filter.drop_duplicates
print(len(camping_filter))
camping_filter.head()

2145

In [84]:
input_file = '../file/20200608' # csv파일들이 있는 디렉토리 위치
output_file = '../file/camping_20200608.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'camping_*.csv'))

allData = []
cols = ['no','camp_nm','hashtag','num_posts','date_time']
for file in allFile_list:
    df = pd.read_csv(file, header=None)
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False, header=None)

In [110]:
# 데이터 불러오기
print(os.getcwd())
path = '../file/camping_20200608.csv'
camping_posts = pd.read_csv(path, index_col=False, names=cols)

/Users/yongjae/Dropbox/layerbylayer/frint


In [111]:
print(len(camping_posts))
print(max(camping_posts.index))

2167
2166


In [112]:
# camping.columns
# camping.head()
camping_posts.sort_values('num_posts', ascending=False)
# camping.

,no,camp_nm,hashtag,num_posts,date_time
1469,1869,산책,산책,6135957,2020-06-08 14:50:46.434935
1995,1729,선인장,선인장,735043,2020-06-08 14:50:46.434935
1767,451,공간,공간,491620,2020-06-08 14:50:46.434935
1184,1234,시크릿,시크릿,216858,2020-06-08 14:50:46.434935
319,619,다온,다온,85458,2020-06-08 14:50:46.434935
1949,283,다온,다온,85458,2020-06-08 14:50:46.434935
1913,1347,힐링캠프,힐링캠프,50619,2020-06-08 14:50:46.434935
1470,1870,부에노스아이레스,부에노스아이레스,39982,2020-06-08 14:50:46.434935
2039,1523,포시즌,포시즌,35583,2020-06-08 14:50:46.434935
1282,1982,포시즌,포시즌,35583,2020-06-08 14:50:46.434935
